In [ ]:
# pip install nvsmi

In [3]:
'''Check GPU Memory Usage'''

import os
import nvsmi
import pandas as pd
import subprocess
import psutil
import psutil

def kill_process(pid):
    try:
        process = psutil.Process(pid)
        process.terminate()  # terminate process
        print(f"Process with PID {pid} terminated successfully.")
    except psutil.NoSuchProcess as e:
        print(f"Process with PID {pid} not found.")
    except psutil.AccessDenied as e:
        print(f"Access to terminate process with PID {pid} denied.")

def get_process_cwd(pid):
    # /proc/{pid}/cwd create path
    proc_path = f"/proc/{pid}/cwd"

    try:
        # Get current working directory via symbolic link
        cwd = os.readlink(proc_path)
        return cwd
    except OSError as e:
#         print(f"Error: {e}")
        return None


process_list = []
for process in nvsmi.get_gpu_processes():
    process_list.append([process.pid,process.gpu_id, process.used_memory, get_process_cwd(process.pid)])
    
gpu_df = pd.DataFrame(process_list,columns=['pid','gpu_id','memory','path']).sort_values(by = "memory",ascending=False)

In [ ]:
gpu_df

In [ ]:
# #kill
# for i in range(len(df)):
#     if "jwkang" in df.iloc[i].path:
#         print(df.iloc[i]['pid'])
#         kill_process(df.iloc[i]['pid'])

In [5]:
'''Check CPU Memory Usage'''
import psutil
import pandas as pd

def get_memory_usage(pid):
    try:
        process = psutil.Process(pid)
        memory_info = process.memory_info()
        # RSS to MB
        return memory_info.rss / (1024 * 1024)
    except psutil.NoSuchProcess as e:
        print(f"Process with PID {pid} not found.")
        return None

def get_all_processes_memory_usage():
    processes = []

    for proc in psutil.process_iter(['pid', 'name']):
        pid = proc.info['pid']
        name = proc.info['name']
        memory_usage = get_memory_usage(pid)

        if memory_usage is not None and get_process_cwd(pid) is not None:
            processes.append({'PID': pid, 'get_process_cwd': get_process_cwd(pid), 'MemoryUsage': memory_usage})

    return processes

# Memory usage dataframe
processes_info = get_all_processes_memory_usage()
memory_df = pd.DataFrame(processes_info)
memory_df = memory_df.sort_values(by='MemoryUsage',ascending=False)

memory_df['MemoryUsage'] = memory_df['MemoryUsage'].apply(lambda x: f'{x:.2f} MB')


In [ ]:
memory_df